In [29]:
%matplotlib inline
import numpy as np
import scipy as sp
import json 
import csv
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [30]:
df=pd.read_json("./data/auctions_detailed.json")
df.head()

,AutoPay,AvailableForPickupDropOff,BestOfferEnabled,BidCount,BuyItNowAvailable,BuyItNowPrice,Charity,ConvertedBuyItNowPrice,ConvertedCurrentPrice,Country,CurrentPrice,DiscountPriceInfo,EligibleForPickupDropOff,EndTime,ExcludeShipToLocation,GalleryURL,GlobalShipping,HandlingTime,HighBidder,HitCount,IntegratedMerchantCreditCardEnabled,ItemID,ListingStatus,ListingType,Location,LotSize,MinimumToBid,NewBestOffer,PaymentAllowedSite,PaymentMethods,PictureURL,PostalCode,PrimaryCategoryID,PrimaryCategoryIDPath,PrimaryCategoryName,ProductID,Quantity,QuantityAvailableHint,QuantitySold,QuantitySoldByPickupInStore,QuantityThreshold,ReserveMet,ReturnPolicy,SKU,Seller,ShipToLocations,Site,StartTime,Storefront,Subtitle,TimeLeft,Title,TopRatedListing,ViewItemURLForNaturalSearch
0,False,NaN,False,0,1,"{u'CurrencyID': u'USD', u'Value': 20.0}",NaN,"{u'CurrencyID': u'USD', u'Value': 20.0}","{u'CurrencyID': u'USD', u'Value': 10.0}",US,"{u'CurrencyID': u'USD', u'Value': 10.0}",NaN,NaN,2015-12-02T19:00:56.000Z,NaN,http://thumbs2.ebaystatic.com/pict/15189544909...,False,2,NaN,23,False,151895449093,Completed,Chinese,"Punta Gorda, Florida",NaN,"{u'CurrencyID': u'USD', u'Value': 10.0}",False,[],[PayPal],[http://i.ebayimg.com/00/s/ODAwWDQ4MA==/z/KsgA...,33983,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'169619285'}",1,NaN,0,0,NaN,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},NaN,"{u'FeedbackRatingStar': u'Yellow', u'UserID': ...","[US, CA]",US,2015-11-25T19:00:56.000Z,NaN,NaN,PT0S,Nokia E Series E63 - Ultramarine blue (Cincinn...,NaN,http://www.ebay.com/itm/Nokia-E-Series-E63-Ult...
1,False,NaN,False,0,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 525.0}",US,"{u'CurrencyID': u'USD', u'Value': 525.0}",NaN,NaN,2015-12-02T22:54:37.000Z,NaN,http://thumbs1.ebaystatic.com/pict/22195115178...,False,2,NaN,54,False,221951151788,Completed,FixedPriceItem,"Ponce, Puerto Rico",NaN,NaN,False,[],[PayPal],[http://i.ebayimg.com/00/s/OTAwWDE0NDA=/z/nT8A...,00732,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'210749029'}",1,NaN,0,0,NaN,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},NaN,"{u'FeedbackRatingStar': u'Turquoise', u'UserID...",[US],US,2015-11-25T22:54:37.000Z,NaN,NaN,PT0S,Samsung Galaxy S6 Edge SM-G925L - 32GB - White...,NaN,http://www.ebay.com/itm/Samsung-Galaxy-S6-Edge...
2,True,NaN,False,0,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 29.95}",US,"{u'CurrencyID': u'USD', u'Value': 29.95}",NaN,NaN,2015-12-02T16:13:29.000Z,NaN,http://thumbs1.ebaystatic.com/pict/22195520835...,True,1,"{u'FeedbackRatingStar': u'None', u'FeedbackPri...",20,False,221955208352,Completed,FixedPriceItem,"Garden Grove, California",NaN,NaN,False,[],[PayPal],[http://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/mQ8A...,92840,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'102630194'}",1,NaN,1,0,NaN,NaN,{u'Refund': u'Money back or exchange (buyer's ...,NaN,"{u'FeedbackRatingStar': u'Red', u'UserID': u'f...",[Worldwide],US,2015-11-30T07:18:25.000Z,{u'StoreURL': u'http://stores.ebay.com/id=8690...,NaN,PT0S,Nokia 2720 - Black (T-Mobile) Cellular Phone,NaN,http://www.ebay.com/itm/Nokia-2720-Black-T-Mob...
3,False,NaN,False,0,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 103.0}",US,"{u'CurrencyID': u'USD', u'Value': 103.0}",NaN,NaN,2015-12-02T17:57:20.000Z,NaN,http://thumbs1.ebaystatic.com/pict/22195095721...,True,1,NaN,159,False,221950957212,Completed,Chinese,"Cleveland, Ohio",NaN,"{u'CurrencyID': u'USD', u'Value': 103.0}",False,[],[PayPal],[http://i.ebayimg.com/00/s/MTQ0MFg4MTA=/z/l00A...,44121,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'177911376'}",1,NaN,0,0,NaN,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},NaN,"{u'FeedbackRatingStar': u'Turquoise', u'UserID...","[US, CA, GB, DK, RO, SK, BG, CZ, FI, HU, LV, L...",US,2015-11-25T17:57:20.000Z,NaN,NaN,PT0S,LG G Fl

**NEXT, we will delete columns that we just don't need to do our analysis. This could even come at the end once we figure out which aren't useful, but must remember to put it here.**


## Now we will do some Feature engineering

This is to figure out which variables actually make a distinction on profitability

In [31]:
df.shape

(9856, 54)

We delete the columns that are irrelevant to our analysis. For example, DiscountPriceInfo tells us no information about our data, because none of our auctions have discounted prices.

In [32]:
del df['Charity'], df['Country'], df['ConvertedCurrentPrice'], df['DiscountPriceInfo']
del df['ListingStatus'], df['PaymentAllowedSite'], df['ReserveMet'], df['SKU'], df['Site'], df['Storefront']
del df['Subtitle'], df['TimeLeft'], df['ViewItemURLForNaturalSearch'], df['ConvertedBuyItNowPrice']

We then remove unnecessary information from each of the columns. For example, on CurrentPrice, we only want the value in the dictionary and not the whole dictionary. 

In [33]:
#df['CurrentPrice'].values[3][u'Value']

for i in df.index:
    
    #getting just the float value and ridding of the dictionary 
    df['CurrentPrice'].values[i] = df['CurrentPrice'].values[i][u'Value']
    
    if pd.isnull(df['BuyItNowPrice'].values[i]) == False:
        #print df['BuyItNowPrice'].values[i]
        df['BuyItNowPrice'].values[i] = df['BuyItNowPrice'].values[i][u'Value']
    if pd.isnull(df['MinimumToBid'].values[i]) == False:
        df['MinimumToBid'].values[i] = df['MinimumToBid'].values[i][u'Value']
    
    #we make a numbering system fo ListingType so we can see which type of auction is most successful
    #if df['ListingType'][i] == 'Chinese':
        #df['ListingType'][i] = 0
    #print df.ix[i,'ListingType']
    
    df['ListingType'] = df['ListingType'].map({'Chinese': 0, 'FixedPriceItem': 1})
    
    #if df.ix[i,'ListingType'] == 'Chinese':
    #    df.ix[i,'ListingType'] == 0
    #df.loc[df['ListingType'] == 'Chinese'] = 0
    #df.loc[df['ListingType'] == 'FixedPriceItem'] = 1
    #elif df['ListingType'][i] == 'FixedPriceItem':
        #df['ListingType'][i] == 1
    #if df.ix[i,'ListingType'] == 'FixedPriceItem':
    #    df.ix[i,'ListingType'] == 1
        
    
    
#because each auction we are dealing with is completed, we say
df=df.rename(columns = {'CurrentPrice':'FinalPrice'})

In [27]:
df.head()

,AutoPay,AvailableForPickupDropOff,BestOfferEnabled,BidCount,BuyItNowAvailable,BuyItNowPrice,FinalPrice,EligibleForPickupDropOff,EndTime,ExcludeShipToLocation,GalleryURL,GlobalShipping,HandlingTime,HighBidder,HitCount,IntegratedMerchantCreditCardEnabled,ItemID,ListingType,Location,LotSize,MinimumToBid,NewBestOffer,PaymentMethods,PictureURL,PostalCode,PrimaryCategoryID,PrimaryCategoryIDPath,PrimaryCategoryName,ProductID,Quantity,QuantityAvailableHint,QuantitySold,QuantitySoldByPickupInStore,QuantityThreshold,ReturnPolicy,Seller,ShipToLocations,StartTime,Title,TopRatedListing
0,False,NaN,False,0,1,20,10,NaN,2015-12-02T19:00:56.000Z,NaN,http://thumbs2.ebaystatic.com/pict/15189544909...,False,2,NaN,23,False,151895449093,Chinese,"Punta Gorda, Florida",NaN,10,False,[PayPal],[http://i.ebayimg.com/00/s/ODAwWDQ4MA==/z/KsgA...,33983,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'169619285'}",1,NaN,0,0,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},"{u'FeedbackRatingStar': u'Yellow', u'UserID': ...","[US, CA]",2015-11-25T19:00:56.000Z,Nokia E Series E63 - Ultramarine blue (Cincinn...,NaN
1,False,NaN,False,0,NaN,NaN,525,NaN,2015-12-02T22:54:37.000Z,NaN,http://thumbs1.ebaystatic.com/pict/22195115178...,False,2,NaN,54,False,221951151788,FixedPriceItem,"Ponce, Puerto Rico",NaN,NaN,False,[PayPal],[http://i.ebayimg.com/00/s/OTAwWDE0NDA=/z/nT8A...,00732,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'210749029'}",1,NaN,0,0,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},"{u'FeedbackRatingStar': u'Turquoise', u'UserID...",[US],2015-11-25T22:54:37.000Z,Samsung Galaxy S6 Edge SM-G925L - 32GB - White...,NaN
2,True,NaN,False,0,NaN,NaN,29.95,NaN,2015-12-02T16:13:29.000Z,NaN,http://thumbs1.ebaystatic.com/pict/22195520835...,True,1,"{u'FeedbackRatingStar': u'None', u'FeedbackPri...",20,False,221955208352,FixedPriceItem,"Garden Grove, California",NaN,NaN,False,[PayPal],[http://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/mQ8A...,92840,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'102630194'}",1,NaN,1,0,NaN,{u'Refund': u'Money back or exchange (buyer's ...,"{u'FeedbackRatingStar': u'Red', u'UserID': u'f...",[Worldwide],2015-11-30T07:18:25.000Z,Nokia 2720 - Black (T-Mobile) Cellular Phone,NaN
3,False,NaN,False,0,NaN,NaN,103,NaN,2015-12-02T17:57:20.000Z,NaN,http://thumbs1.ebaystatic.com/pict/22195095721...,True,1,NaN,159,False,221950957212,Chinese,"Cleveland, Ohio",NaN,103,False,[PayPal],[http://i.ebayimg.com/00/s/MTQ0MFg4MTA=/z/l00A...,44121,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'177911376'}",1,NaN,0,0,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},"{u'FeedbackRatingStar': u'Turquoise', u'UserID...","[US, CA, GB, DK, RO, SK, BG, CZ, FI, HU, LV, L...",2015-11-25T17:57:20.000Z,LG G Flex D959 - 32GB - Titan Silver (T-Mobile...,NaN
4,False,NaN,True,0,NaN,NaN,150,NaN,2015-12-02T14:09:57.000Z,NaN,http://thumbs1.ebaystatic.com/pict/18194779804...,True,2,"{u'FeedbackRatingStar': u'Turquoise', u'Feedba...",62,False,181947798040,FixedPriceItem,"Davis, California",NaN,NaN,False,[PayPal],[http://i.ebayimg.com/00/s/MTYwMFgxMTk1/z/N6kA...,95616,9355,15032:9355,Cell Phones & Accessories:Cell Phones & Smartp...,"{u'Type': u'Reference', u'Value': u'170048134'}",1,NaN,1,0,NaN,{u'ReturnsAccepted': u'ReturnsNotAccepted'},"{u'FeedbackRatingStar': u'Purple', u'UserID': ...","[US, CA, GB, DK, RO, SK, BG, CZ, FI, HU, LV, L...",2015-12-01T07:02:11.000Z,iPhone 5s - 16GB - Gold (AT&T) Smartphone BAD...,NaN


In [17]:
dftouse = df.copy()

**We can now make some separator variables to make listings unique**

Certain parameters that are zero are can be significant indicators of specific characteristics of an auction. For example, if the HitCount is zero, that means nobody really bothered to interact with the auction at all.

In [18]:
ZERO_IMPORTANT_VARIABLES = ['BidCount', 'HitCount']

To mark binary or indicator variables, we see which of the dataframe fall into this category

In [19]:
for v in df.columns:
    l=df[v].unique()
    if len(l) <= 10:
        print v, l

AutoPay [False True]
AvailableForPickupDropOff [ nan   1.]
BestOfferEnabled [False True]
BuyItNowAvailable [  1.  nan]
EligibleForPickupDropOff [ nan   1.]


TypeError: unhashable type: 'list'